<a href="https://colab.research.google.com/github/RocketRikky/Secure_and_Private_AI/blob/master/Pysyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch as th

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [0]:
y

tensor([ 2,  4,  6,  8, 10])

In [0]:
!pip install syft

In [0]:
import syft as sy

In [0]:
hook = sy.TorchHook(th)

W0821 13:35:19.847989 140548966532992 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Basic Remote Execution in Pysyft

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
bob._objects

{2208949348: tensor([[-0.9050],
         [-1.0621]], grad_fn=<AddmmBackward>), 33338848054: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 79063662747: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [0]:
bob._objects

{2208949348: tensor([[-0.9050],
         [-1.0621]], grad_fn=<AddmmBackward>), 33338848054: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 64336749461: tensor([1, 2, 3, 4, 5]), 79063662747: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

In [0]:
x.location == bob

True

In [0]:
x.id_at_location

64336749461

In [0]:
x.id

83906525857

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[PointerTensor | me:83906525857 -> bob:64336749461]

In [0]:
x=x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{2208949348: tensor([[-0.9050],
         [-1.0621]], grad_fn=<AddmmBackward>), 33338848054: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 79063662747: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

Playing with Remote Tensors

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
x = x.send(bob, alice)

In [0]:
bob._objects

{2208949348: tensor([[-0.9050],
         [-1.0621]], grad_fn=<AddmmBackward>), 33338848054: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 67142272510: tensor([1, 2, 3, 4, 5]), 79063662747: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

In [0]:
alice._objects

{25474947992: tensor([1, 2, 3, 4, 5]), 28712763890: tensor([[1.],
         [1.]], requires_grad=True), 83490191520: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:58857338023 -> bob:67142272510]
	-> [PointerTensor | me:86081660378 -> alice:25474947992]

In [0]:
x = x.get()
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
bob._objects

{2208949348: tensor([[-0.9050],
         [-1.0621]], grad_fn=<AddmmBackward>), 33338848054: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 79063662747: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

In [0]:
alice._objects

{28712763890: tensor([[1.],
         [1.]], requires_grad=True), 83490191520: tensor([[1., 1.],
         [0., 1.]], requires_grad=True)}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [0]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:20530110244 -> bob:44266323182]

In [0]:
y

(Wrapper)>[PointerTensor | me:86429986957 -> bob:18909283708]

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:2497383500 -> bob:64367274909]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:6748557937 -> bob:76459592540]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:5997317246 -> bob:77569244872]

In [0]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

Learn a Simple Linear Model

In [0]:
input1 = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
input1

(Wrapper)>[PointerTensor | me:90918316734 -> bob:32931594907]

In [0]:
target

(Wrapper)>[PointerTensor | me:2426828801 -> bob:83163089529]

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [0]:
for i in range(10):  
  
  pred = input1.mm(weights)

  loss = ((pred - target)**2).sum()
  
  loss.backward()
  
  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0

  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


Garbage Collection and Common Errors

In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{82234136843: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x.child.garbage_collect_data

True

In [0]:
bob._objects

{25367694008: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:64614488625 -> bob:87548578120]

In [0]:
bob._objects

{87548578120: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{87548578120: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{87548578120: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
  x = th.tensor([1,2,3,4,5]).send(bob)

Toy Federated Learning

In [0]:
from torch import nn, optim

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [0]:
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
  for iter in range(iterations):
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()
    opt.step()
    print(loss.data)
    
train()

tensor(4.7044)
tensor(1.1920)
tensor(0.6557)
tensor(0.4344)
tensor(0.2966)
tensor(0.2046)
tensor(0.1423)
tensor(0.0997)
tensor(0.0704)
tensor(0.0501)
tensor(0.0360)
tensor(0.0260)
tensor(0.0189)
tensor(0.0138)
tensor(0.0102)
tensor(0.0075)
tensor(0.0056)
tensor(0.0042)
tensor(0.0031)
tensor(0.0023)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
_data, _target = datasets[0]
_data

(Wrapper)>[PointerTensor | me:79901901309 -> bob:40670496069]

In [0]:
def train(iterations=20):
  
  model = nn.Linear(2,1)
  opt = optim.SGD(params=model.parameters(), lr=0.1)
  
  for iter in range(iterations):

    for _data, _target in datasets:
      model = model.send(_data.location)

      opt.zero_grad()

      pred = model(_data)

      loss = ((pred - _target)**2).sum()

      loss.backward()

      opt.step()

      model = model.get()
  
      print(loss.get())

In [0]:
train()

tensor(2.3097, requires_grad=True)
tensor(0.1674, requires_grad=True)
tensor(0.1258, requires_grad=True)
tensor(0.1046, requires_grad=True)
tensor(0.0871, requires_grad=True)
tensor(0.0725, requires_grad=True)
tensor(0.0603, requires_grad=True)
tensor(0.0502, requires_grad=True)
tensor(0.0418, requires_grad=True)
tensor(0.0348, requires_grad=True)
tensor(0.0289, requires_grad=True)
tensor(0.0241, requires_grad=True)
tensor(0.0200, requires_grad=True)
tensor(0.0167, requires_grad=True)
tensor(0.0139, requires_grad=True)
tensor(0.0116, requires_grad=True)
tensor(0.0096, requires_grad=True)
tensor(0.0080, requires_grad=True)
tensor(0.0067, requires_grad=True)
tensor(0.0055, requires_grad=True)
tensor(0.0046, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0027, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0011, requi

In [0]:
#list(model.parameters())

Advanced Remote Execution Tools